The following pip installs are necessary in whatever environment you choose to work in:

youtube_transcript_api
googleapiclient
pandas
os
pickle
langchain
openai
chromadb
langchain-openai



Initialise and Set up Google API Key

In [15]:
import pickle
import os
import pandas as pd
from googleapiclient.discovery import build
from googleapiclient.errors import HttpError
from youtube_transcript_api import YouTubeTranscriptApi


f = open("keys/google_key.txt", "r")
key = f.readlines()[0]
f.close()

youtube = build('youtube', 'v3', developerKey=key)

import os

f = open("keys/openai_key.txt", "r")
key = f.readlines()[0]
f.close()

os.environ["OPENAI_API_KEY"] = key      # LangChain requires API key in environment





User Inputs (prompt)

In [3]:
product = "Apple Watch Series 9"

In [4]:
search_terms = f"{product} Review"    # insert prompt -> "Brand Product Model"
max_result = 10                         # No. of results (1-50)


In [36]:
from openai import OpenAI
import ast

client = OpenAI()

prompt =  f"Identify 3 main competitors for {product}. Output the results in the following format: ['a', 'b', 'c'] that can immediately be used as code," \
          " where a, b, and c are each one key design feature. "


chat_completion = client.chat.completions.create(
    model="gpt-3.5-turbo",
    messages=[{"role": "user", "content": prompt,}],
    temperature=0,
)



competitors = ast.literal_eval(chat_completion.choices[0].message.content)

print(competitors)

reviewlist = []

for i in range(len(competitors)):
    reviewlist.append(competitors[i] + " Review")

competitors.append(product)
reviewlist.append(search_terms)
print(competitors)
print(reviewlist)

['Samsung Galaxy Watch 4', 'Fitbit Sense', 'Garmin Forerunner 945']
['Samsung Galaxy Watch 4', 'Fitbit Sense', 'Garmin Forerunner 945', 'Apple Watch Series 9']
['Samsung Galaxy Watch 4 Review', 'Fitbit Sense Review', 'Garmin Forerunner 945 Review', 'Apple Watch Series 9 Review']


Define containers to store info

In [14]:
vid_id = []             	  # video id
vid_page = []       		    # video links (https...)
vid_title = []              # video title
num_comments = []           # official number of comments
load_error = 0              # error counter
can_load_title = []         # temp. list for storing title w/o loading error
can_load_page = []          # temp. list for storing links w/o loading error
num_page = []               # comment_response page number
page_title = []             # comment_response video title
comment_resp = []           # comment_response
comment_list = []           # temp. list for storing comments
comment_data = []           # comments & replies from comment_response
all_count = 0               # total number of comments


Search for Video IDs based on User Inputs

In [16]:
print("Search for Videos IDs...")

vid_id = []

for competitor in competitors:
    try:
            search_response = youtube.search().list(
                q=competitor,
                part='id',
                type='video',
                maxResults=max_result
            ).execute()

            for item in search_response['items']:
                vid_id.append(item['id']['videoId'])

    except HttpError as e:
        print('An HTTP error %d occurred:\n%s' % (e.resp.status, e.content))

    



Search for Videos IDs...


Use the list of Video IDs to get video data

In [18]:
print("Get video data...")
for i in range(len(vid_id)):
    request = youtube.videos().list(
        part="snippet, statistics",
        id=vid_id[i]
        )
    video_response = request.execute()
    print(video_response)

    title = video_response['items'][0]['snippet']['title']
    vid_title.append(title)
    try:                        # use try/except as some videos might not load
        comment_count = video_response['items'][0]['statistics']['commentCount']
        print("Video", i + 1, "-", title, "-- Comment count: ", comment_count)
        print()
        num_comments.append(comment_count)
    except:
        print("Video", i + 1, "-", title, "-- Comments are turned off")
        print()
        num_comments.append(0)


Get video data...
{'kind': 'youtube#videoListResponse', 'etag': 'frwQiiaEWPxra4PhgKHjZQOmnqk', 'items': [{'kind': 'youtube#video', 'etag': 'HsEFdNOrxSFZTphQISAG3K_jtn8', 'id': 'E5DlpONIW5M', 'snippet': {'publishedAt': '2021-08-18T14:30:02Z', 'channelId': 'UCddiUEpeqJcYeBxX1IVBKvQ', 'title': 'Samsung Galaxy Watch 4 review: gone Google', 'description': "Samsung has released the Galaxy Watch 4 and Watch 4 Classic. They run Google's Wear OS 3 platform, but they're Samsung watches through and through. Dieter Bohn reviews the watches and explains what's it's like to finally have a good smartwatch for Android users… with Bixby.\n\nIf you buy something from a Verge link, Vox Media may earn a commission. See our ethics policy: http://bit.ly/2TQsqjB\n\nYou can get the Galaxy Watch 4 here: https://shop-links.co/1749444477664951566\n\nShop The Verge's merch here: https://shop.theverge.com/ \nSubscribe: http://goo.gl/G5RXGs\nLike The Verge on Facebook: https://goo.gl/2P1aGc\nFollow on Twitter: http

Show videos without loading errors

In [20]:
print("Videos that can load...")
vid_page = can_load_page                    # update vid_page with those with no load error
vid_title = can_load_title                  # update vid_title with those with no load error
for i in range(len(vid_title)):
    if vid_title[i] == 'YouTube':           # default error title is 'YouTube'
        vid_title[i] = 'Video_' + str(i+1)  # replace 'YouTube' with Video_1 format
    print(i + 1, vid_title[i])


Videos that can load...


In [21]:
def save_transcript_to_file(video_id, output_folder):
    try:
        # Get the transcript for the video
        transcript = YouTubeTranscriptApi.get_transcript(video_id)

        # Extract the text from the transcript
        text = ''
        for segment in transcript:
            text += segment['text'] + ' '

        # Create the output folder if it doesn't exist
        if not os.path.exists(output_folder):
            os.makedirs(output_folder)

        # Save the text to a file
        output_file = os.path.join(output_folder, f"{video_id}_transcript.txt")
        with open(output_file, 'w', encoding='utf-8') as file:
            file.write(text)

        
        print("Transcript saved successfully!")
    except Exception as e:
        print("An error occurred:", str(e))


for video_id in vid_id:
    save_transcript_to_file(video_id, "transcript_data")
# Example usage
# video_id = '5KfXKQOmqY8'  # Replace 'VIDEO_ID' with the actual ID of the YouTube video
# output_file = 'transcript.txt'

# save_transcript_to_file(video_id, output_file)


Transcript saved successfully!
Transcript saved successfully!
Transcript saved successfully!
Transcript saved successfully!
Transcript saved successfully!
Transcript saved successfully!
Transcript saved successfully!
Transcript saved successfully!
Transcript saved successfully!
Transcript saved successfully!
Transcript saved successfully!
An error occurred: 
Could not retrieve a transcript for the video https://www.youtube.com/watch?v=joYeeOVHjOM! This is most likely caused by:

No transcripts were found for any of the requested language codes: ('en',)

For this video (joYeeOVHjOM) transcripts are available in the following languages:

(MANUALLY CREATED)
None

(GENERATED)
 - hi ("Hindi (auto-generated)")[TRANSLATABLE]

(TRANSLATION LANGUAGES)
 - af ("Afrikaans")
 - ak ("Akan")
 - sq ("Albanian")
 - am ("Amharic")
 - ar ("Arabic")
 - hy ("Armenian")
 - as ("Assamese")
 - ay ("Aymara")
 - az ("Azerbaijani")
 - bn ("Bangla")
 - eu ("Basque")
 - be ("Belarusian")
 - bho ("Bhojpuri")
 - bs 

Create directory

In [22]:
try:                                              # Create directory named after search terms
    os.makedirs("support/%s" % search_terms)
    print("Directory", search_terms, "created")
except FileExistsError:
    print("Directory", search_terms, "exists")

try:                                              # Create directory to store current search terms
    os.makedirs("support/_current_")
    print("Directory _current_ created")
except FileExistsError:
    print("Directory _current_ exists")


Directory Apple Watch Series 9 Review exists
Directory _current_ exists


Save files for future use

In [23]:


pickle.dump(search_terms, open("support/%s/searchTerms.pkl" % search_terms, "wb"))
# pickle.dump(comment_list, open("support/%s/comment_list.pkl" % search_terms, "wb"))
pickle.dump(vid_title, open("support/%s/vid_title.pkl" % search_terms, "wb"))
pickle.dump(vid_page, open("support/%s/vid_page.pkl" % search_terms, "wb"))
pickle.dump(vid_id, open("support/%s/vid_id.pkl" % search_terms, "wb"))


Save files for next step

In [24]:
import shutil

# source = "support/%s/comments.txt" % search_terms
# destination = "support/_current_/comments.txt"
# shutil.copyfile(source, destination)

pickle.dump(search_terms, open("support/_current_/searchTerms.pkl", "wb"))


Clear data for rerun

In [25]:
comment_list.clear()
vid_title.clear()
vid_page.clear()
vid_id.clear()


Load Private Documents


In [26]:
from langchain_community.document_loaders import DirectoryLoader

from langchain.document_loaders import TextLoader



loader = DirectoryLoader('transcript_data/', glob="./*.txt", loader_cls=TextLoader)

document = loader.load()

Split documents into smaller parts


In [27]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000,
    chunk_overlap=200
)
splits = text_splitter.split_documents(document)

print("Number of splits in document loaded:", len(splits))


Number of splits in document loaded: 628


Use openAI Embeddings


In [28]:

from langchain_openai import OpenAIEmbeddings
embedding = OpenAIEmbeddings()


Remove 'persist' directory, setup folder


In [29]:
import shutil
import pandas as pd
search_terms = pd.read_pickle("support/_current_/searchTerms.pkl")
print("Processing folder:", search_terms)

try:
    shutil.rmtree('support/%s/persist' % search_terms)       # remove old version
    print("Deleting previous store")
except:
    print("No store found")

persist_directory = 'support/%s/persist' % search_terms     # create new version

files = os.listdir("support/%s" % search_terms)             # list all files
print("Files in folder:", files)


Processing folder: Apple Watch Series 9 Review
Deleting previous store
Files in folder: ['searchTerms.pkl', 'vid_id.pkl', 'vid_page.pkl', 'vid_title.pkl']


In [30]:
from langchain.vectorstores import Chroma

vectordb = Chroma.from_documents(
    documents=splits,                           # target the splits created from the documents loaded
    embedding=embedding,                        # use the OpenAI embedding specified
    persist_directory=persist_directory         # store in the persist directory for future use
)

vectordb.persist()                              # store vectordb

print("Persist Directory created.")
print("Size of Vector Database:", vectordb._collection.count())    # same as the number of splits


Persist Directory created.
Size of Vector Database: 628


Retrieve vectordb


In [31]:
import pandas as pd
search_terms = pd.read_pickle("support/_current_/searchTerms.pkl")

from langchain_openai import OpenAIEmbeddings
embedding = OpenAIEmbeddings()

from langchain.vectorstores import Chroma
persist_directory = 'support/%s/persist' % search_terms

vectordb = Chroma(
    persist_directory=persist_directory,
    embedding_function=embedding
    )

print("Processing folder:", search_terms)
print("Size of Vector Database", vectordb._collection.count())    # same as before


Processing folder: Apple Watch Series 9 Review
Size of Vector Database 628


In [32]:
from langchain_openai import ChatOpenAI
llm = ChatOpenAI(model_name="gpt-4", temperature=0)               # gpt model can be changed

In [44]:
##Cite sources
def process_llm_response(llm_response):
    print(llm_response['result'])
    print('\n\nSources:')
    for source in llm_response["source_documents"]:
        print(source.metadata['source'])

competitorstring = ""

for i in range(len(competitors) - 1):
    competitorstring = competitorstring + ", " + competitors[i]

competitorstring = competitorstring[1:]

print(competitors)

print(competitorstring)
        

['Samsung Galaxy Watch 4', 'Fitbit Sense', 'Garmin Forerunner 945', 'Apple Watch Series 9']
 Samsung Galaxy Watch 4, Fitbit Sense, Garmin Forerunner 945


In [47]:
from langchain.chains import RetrievalQA

qa_chain = RetrievalQA.from_chain_type(
    llm,
    # retriever=vectordb.as_retriever(search_type="mmr", search_kwargs={"k": 4, "fetch_k": 6}),
    # retriever=vectordb.as_retriever(search_type="similarity", search_kwargs={"k": 4}),
    retriever=vectordb.as_retriever(),
    return_source_documents=True
    )




question = f"Tell me some insights with regard to the Apple Watch Series 9's following categories: Positively reviewed components, negatively reviewed components, places for improvement, and a general summary. After giving me that information, identify design opportunities and provide me with recommendations."  # input question


result = qa_chain({"query": question})
process_llm_response(result)

# print("\n--Results with metadata--")
# print(result)
# print("\n--The prompt--")
# print(result["query"])
# print("\n--The sources--")
# print(result["source_documents"])
# print("\n--The final response--")
# print(result["result"])

Positively Reviewed Components:
1. The Apple Watch Series 9 has been praised for its accuracy in many health tracking categories, including heart rate measurement, sleep stage tracking, GPS tracking consistency, and oxygen saturation measurements. 
2. It has a high correlation value with the reference device, placing it among the better performing watches in comparison tests. 
3. It has a ton of great features, especially for iPhone users.

Negatively Reviewed Components:
1. The Apple Watch Series 9 has some issues with software updates, particularly for users who wear the watch to bed and thus can't have it on the charger for automatic updates at 2 a.m.
2. The watch's repairability is a downside, particularly when it comes to replacing the battery.

Places for Improvement:
1. The watch could be improved by allowing users to set when automatic updates should install.
2. The repairability of the watch, particularly the battery, could be improved to make it more user-friendly.

General S

In [48]:
from langchain.chains import create_history_aware_retriever
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

retriever = vectordb.as_retriever()

contextualize_q_system_prompt = """Given a chat history and the latest user question \
which might reference context in the chat history, formulate a standalone question \
which can be understood without the chat history. Do NOT answer the question, \
just reformulate it if needed and otherwise return it as is."""
contextualize_q_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", contextualize_q_system_prompt),
        MessagesPlaceholder("chat_history"),
        ("human", "{input}"),
    ]
)
history_aware_retriever = create_history_aware_retriever(
    llm, retriever, contextualize_q_prompt
)

In [49]:
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain

qa_system_prompt = """You are an assistant for question-answering tasks. \
Use the following pieces of retrieved context to answer the question. \
If you don't know the answer, just say that you don't know. \
Use three sentences maximum and keep the answer concise.\

{context}"""
qa_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", qa_system_prompt),
        MessagesPlaceholder("chat_history"),
        ("human", "{input}"),
    ]
)


question_answer_chain = create_stuff_documents_chain(llm, qa_prompt)

rag_chain = create_retrieval_chain(history_aware_retriever, question_answer_chain)

In [50]:
from langchain_core.messages import HumanMessage

chat_history = []

question = "What is the apple watch good at??"
ai_msg_1 = rag_chain.invoke({"input": question, "chat_history": chat_history})
chat_history.extend([HumanMessage(content=question), ai_msg_1["answer"]])

second_question = "What are its competitors?"
ai_msg_2 = rag_chain.invoke({"input": second_question, "chat_history": chat_history})

print(ai_msg_1["answer"])

The Apple Watch has several competitors in the smartwatch market. These include devices like the Garmin watches, which are often more expensive but are known for their fitness tracking capabilities. Other competitors include the Fitbit devices, the Aura Ring, Whoop Straps, and the Withings Sleep Analyzer. There are also smartwatches from other tech companies like Samsung and Google.


In [53]:
from langchain.chains import create_history_aware_retriever
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.runnables import RunnablePassthrough
from langchain_core.runnables.history import RunnableWithMessageHistory
from langchain_community.chat_message_histories import ChatMessageHistory

retriever = vectordb.as_retriever()

### Contextualize question ###
contextualize_q_system_prompt = """Given a chat history and the latest user question \
which might reference context in the chat history, formulate a standalone question \
which can be understood without the chat history. Do NOT answer the question, \
just reformulate it if needed and otherwise return it as is."""
contextualize_q_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", contextualize_q_system_prompt),
        MessagesPlaceholder("chat_history"),
        ("human", "{input}"),
    ]
)
history_aware_retriever = create_history_aware_retriever(
    llm, retriever, contextualize_q_prompt
)


### Answer question ###
qa_system_prompt = """You are an assistant for question-answering tasks. \
Use the following pieces of retrieved context to answer the question. \
If you don't know the answer, just say that you don't know. \
Use three sentences maximum and keep the answer concise.\

{context}"""
qa_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", qa_system_prompt),
        MessagesPlaceholder("chat_history"),
        ("human", "{input}"),
    ]
)
question_answer_chain = create_stuff_documents_chain(llm, qa_prompt)

rag_chain = create_retrieval_chain(history_aware_retriever, question_answer_chain)


### Statefully manage chat history ###
store = {}


def get_session_history(session_id: str) -> BaseChatMessageHistory:
    if session_id not in store:
        store[session_id] = ChatMessageHistory()
    return store[session_id]


conversational_rag_chain = RunnableWithMessageHistory(
    rag_chain,
    get_session_history,
    input_messages_key="input",
    history_messages_key="chat_history",
    output_messages_key="answer",
)

In [54]:
conversational_rag_chain.invoke(
    {"input": "Tell me some insights with regard to the Apple Watch Series 9's following categories: Positively reviewed components, negatively reviewed components, places for improvement, and a general summary. After giving me that information, identify design opportunities and provide me with recommendations."},
    config={
        "configurable": {"session_id": "abc123"}
    },  # constructs a key "abc123" in `store`.
)["answer"]

"Positively reviewed components of the Apple Watch Series 9 include its accuracy in health tracking categories and its brighter screen. It also has a more powerful S9 chip which enables new features like the double tap. Negatively reviewed components include its compatibility, as it doesn't work well with Android phones. There are also issues with software updates and repairability, particularly the inability to replace the battery without professional help.\n\nAreas for improvement include offering more flexibility with software updates, improving compatibility with non-Apple devices, and enhancing repairability, especially regarding battery replacement. In general, the Apple Watch Series 9 is similar to the Series 8 but with a few improvements, and it performs well among other watches in the market.\n\nDesign opportunities could include creating a user-friendly battery replacement process, improving software update flexibility, and enhancing compatibility with Android devices. Recomm

In [55]:
conversational_rag_chain.invoke(
    {"input": "Who are the apple watch's competitors, based on your context?"},
    config={"configurable": {"session_id": "abc123"}},
)["answer"]

'Based on the context, the competitors of the Apple Watch include selected Huawei watches, Google Pixel watches, and some Galaxy watches. Another competitor mentioned is the Coros watch.'

In [56]:
conversational_rag_chain.invoke(
    {"input": f"For the each of the {competitors}, tell me their positively reviewed components, negatively reviewed components and places for improvement."},
    config={"configurable": {"session_id": "abc123"}},
)["answer"]

"I'm sorry, but the context provided does not contain specific information about the positively reviewed components, negatively reviewed components, and places for improvement for the Fitbit Sense, Garmin Forerunner 945, and Apple Watch Series 9. However, for the Samsung Galaxy Watch 4, the positively reviewed components include its high-quality OLED display, unique body composition measurements, and smooth performance. Negatively, it does not pair with iOS devices. A place for improvement could be increasing its compatibility with non-Samsung devices."

In [57]:
conversational_rag_chain.invoke(
    {"input":"Tell me some insights with regard to the Samsung Galaxy Watch 4's following categories: Positively reviewed components, negatively reviewed components, places for improvement, and a general summary. After giving me that information, identify design opportunities and provide me with recommendations."},
    config={"configurable": {"session_id": "abc123"}},
)["answer"]

"The Samsung Galaxy Watch 4 has been positively reviewed for its high-quality OLED display, unique body composition measurements, and smooth performance. It's also appreciated for its comprehensive array of fitness tracking features and solid functionality. \n\nOn the downside, the watch does not pair with iOS devices, which limits its user base. \n\nA place for improvement could be increasing its compatibility with non-Samsung devices, particularly iOS, to broaden its appeal to a wider range of consumers.\n\nIn summary, the Samsung Galaxy Watch 4 is a high-quality smartwatch with plenty of features that make it popular among Android users. However, its lack of compatibility with iOS devices is a significant drawback.\n\nDesign opportunities could include working on its compatibility with iOS devices. This would make the watch more versatile and appealing to a broader market. \n\nAs for recommendations, if you're an Android user, particularly a Samsung user, the Galaxy Watch 4 is a gre

In [58]:
conversational_rag_chain.invoke(
    {"input": "How can this understanding help me to design a better apple watch?"},
    config={"configurable": {"session_id": "abc123"}},
)["answer"]

"Understanding the strengths and weaknesses of the Samsung Galaxy Watch 4 can provide valuable insights for designing a better Apple Watch. For instance, the Galaxy Watch 4's high-quality OLED display and unique body composition measurements are features that users appreciate, so ensuring the Apple Watch matches or exceeds these features could be beneficial. \n\nThe Galaxy Watch 4's lack of compatibility with iOS devices is a significant drawback. Ensuring that the Apple Watch maintains strong compatibility with both iOS and potentially Android devices could give it a competitive edge.\n\nAdditionally, understanding the importance of comprehensive fitness tracking features and solid functionality can guide the development of new features for the Apple Watch. \n\nFinally, user feedback on the Galaxy Watch 4's areas for improvement, such as better touch sensitivity and preventing accidental taps, can be used to refine the user interface and touch responsiveness of the Apple Watch."

In [59]:
conversational_rag_chain.invoke(
    {"input":"Tell me some insights with regard to the Garmin 945's following categories: Positively reviewed components, negatively reviewed components, places for improvement, and a general summary. After giving me that information, identify design opportunities and provide me with recommendations."},
    config={"configurable": {"session_id": "abc123"}},
)["answer"]

"The Garmin Forerunner 945 has been positively reviewed for its slim, attractive design and the impressive number of sensors packed into its body. It's also appreciated for its wide range of features, including support for over 30 indoor and outdoor sports, and a heart rate monitor that takes measurements 24/7. \n\nNegatively reviewed components are not explicitly mentioned in the provided context. However, potential areas for improvement could include enhancing the screen resolution beyond 240 by 240 pixels, and possibly adding more smartwatch features to compete with other high-end smartwatches.\n\nIn summary, the Garmin Forerunner 945 is a top-of-the-line model that is particularly suited for athletes, especially triathletes, due to its compact, lightweight design and extensive sports and fitness tracking features.\n\nDesign opportunities could include improving the screen resolution for a more detailed and vibrant display, and expanding the smartwatch features to make it more versa

In [60]:
conversational_rag_chain.invoke(
    {"input":"Tell me some insights with regard to the Fitbit Sense following categories: Positively reviewed components, negatively reviewed components, places for improvement, and a general summary. After giving me that information, identify design opportunities and provide me with recommendations."},
    config={"configurable": {"session_id": "abc123"}},
)["answer"]

"The Fitbit Sense has been positively reviewed for its focus on both physical and mental well-being, its sleep tracking feature, and its wide range of workout types. It's also appreciated for its design, which is inspired by the lines of the human body.\n\nNegatively reviewed components include the stress tracking feature, which doesn't feel fully fledged yet. \n\nPlaces for improvement could include further development of the stress tracking feature to provide more accurate and useful data, and possibly enhancing the smartwatch features to compete with other high-end smartwatches.\n\nIn summary, the Fitbit Sense is a high-end smartwatch that offers a wide range of health and fitness tracking features, with a particular emphasis on mental well-being. However, some features like stress tracking need further development.\n\nDesign opportunities could include improving the stress tracking feature and expanding the smartwatch features to make it more versatile and appealing to a broader ra

In [61]:
conversational_rag_chain.invoke(
    {"input":"Taking into account the prior analysis of the Samsung Galaxy Watch 4, Garmin Forerunner 945 and Fitbit Sense, give me quantifiable design recommendations to improve the Apple Watch Series 9"},
    config={"configurable": {"session_id": "abc123"}},
)["answer"]

"1. Display Quality: The Samsung Galaxy Watch 4's high-quality OLED display is a standout feature. Apple could consider enhancing the resolution of the Apple Watch Series 9 to match or exceed this.\n\n2. Fitness Tracking: The Garmin Forerunner 945's extensive sports and fitness tracking features are highly appreciated. Apple could consider expanding the range of workout types supported by the Apple Watch Series 9.\n\n3. Mental Well-being Features: The Fitbit Sense's focus on mental well-being is unique. Apple could consider adding features that monitor and support mental health, such as stress or mood tracking.\n\n4. Compatibility: The Samsung Galaxy Watch 4's lack of compatibility with iOS is a drawback. Apple could consider enhancing the compatibility of the Apple Watch Series 9 with Android devices to broaden its user base.\n\n5. Battery Life: All three watches have been noted for their good battery life. Apple could consider improving the battery life of the Apple Watch Series 9 to